# Financial Markets - Practical Lesson 6

Last week we looked at how to extract ('bootstrap' or 'calibrate') a discount curve from observed market quotes.

This week we're going to move onto using that discount curve to price EURIBOR/LIBOR interest rate swaps and swaptions.

### Refresher

Let's start by reminding ourselves of the differences and similarities between the EONIA/OIS and EURIBOR/LIBOR interest rate markets.

* Similarity: both are **unsecured** lending, i.e. the lender assumes the risk of losing the capital if the borrower fails during the lending period.
* Similarity: both represent some kind of 'average' interest rate between similar, large financial institutions.
* Difference:
    * EONIA/OIS are related to **overnight** lending, which needs to be renewed each day. This means that each day the lender can choose to *not* renew the loan or lend the capital to a different borrower.
    * EURIBOR/LIBOR refer to **term** lending, i.e. lending periods such as one month, three months, etc. The lender needs to wait until the expiration of the loan before having the option to lend the capital to a different borrower.
* Difference:
    * EONIA/OIS rates are a volume-weighted average of real transactions
    * EURIBOR/LIBOR are the determined through a survey of panel banks' percieved market rates
    
A naive understanding of the interest rate markets would lead one to believe that these different markets (O/N, 1M, 3M, etc) can all be priced with a single discount curve - indeed in periods of low market stress, this has been the case.

In reality, the details of the liquidity and counterparty risk involved in each type of transaction are such that there is a *basis* between these markets, and therefore each one has a different discount (or rate) curve associated with it.

### EURIBOR/LIBOR swaps and swaptions

Given the fact that LIBOR curves do not represent a 'pure' interest rate market but incorporate elements of liquidity and counterparty risk (not to mention the various rate-fixing collusion scandals that have happened over the years), it is surprising that, so many years after the financial crisis, they still maintain so much importance as benchmark rate markets for a wide range of purposes.

Indeed, though OIS markets are fully liquid enough to extract information about forward interest rates, there is no functioning options market from which to extract information about the volatility of those interest rates. The liquidity in rate volatility continues to be present only in the LIBOR swaptions markets.

As such, it continues to be important for banks to be able to price and calibrate market parameters against LIBOR instruments.

### Structure of today's lesson

We'll start writing a class which represents and interest rate swap (IRS) on a LIBOR index. The class will have a method which, given a discount curve and a forward rate curve, will calculate the NPV of the swap. The latter curve will be used for determining the forward rates to **calculate** the expected value of the floating leg cash flows, whereas the former curve will be used to **discount** both the floating and fixed leg cash flows.

Next we'll write a similar class for interest rate swaptions. We'll add a method for calculating the NPV analytically using the Black-Scholes formula, and then an additional method for calculating the same NPV via a Monte-Carlo simulation.

## Interest Rate Swaps

Interest rate swaps consist of a floating leg and a fixed leg. The contract parameters are:

* Start date $d_0$
* Notional $N$
* Fixed rate $K$
* Floating rate tenor (months)
* Maturity (years)

The floating leg pays the reference LIBOR fixing at a frequency equal to the tenor of the floating rate - so for example an IRS on a 3-month LIBOR will pay a floating coupon every three months, an IRS on 6-month EURIBOR pays the floating coupon every six months and so on.

The fixed leg pays a predetermined cash flow at annual frequency, regardless of the tenor of the underlying floating rate. We will only consider swaps with maturities which are multiples of 1 year.

We can modify the function in `finmarkets.py` to generate the payment dates for both the fixed and floating legs, as follows:

In [1]:
from datetime import date
from dateutil.relativedelta import relativedelta

def generate_swap_dates(start_date, n_months, tenor_months):
    
    dates = []
    
    for n in range(0, n_months, tenor_months):
        dates.append(start_date + relativedelta(months=n))
    
    dates.append(start_date + relativedelta(months=n_months))
    
    return dates

generate_swap_dates(date.today(), 24, 3)

[datetime.date(2019, 10, 24),
 datetime.date(2020, 1, 24),
 datetime.date(2020, 4, 24),
 datetime.date(2020, 7, 24),
 datetime.date(2020, 10, 24),
 datetime.date(2021, 1, 24),
 datetime.date(2021, 4, 24),
 datetime.date(2021, 7, 24),
 datetime.date(2021, 10, 24)]

Using this function and the contract parameters we can determine a sequence of payment dates for each of the two legs.

$
\def\NPV{\mathrm{NPV}}
\def\float{\mathrm{float}}
\def\fixed{\mathrm{fixed}}
$
Let $d_0 = d_0^\fixed, ..., d_p^\fixed$ be the fixed leg payment dates and $d_0 = d_0^\float, ..., d_q^\float$ be the floating leg payment dates.

Also let's use the following notation:

* $d$ - the pricing date
* $P(d, d')$  - the discount factor observed in date $d$ for value date $d'$
* $F(d, d', d'')$ - the forward rate observed in date $d$ for the period $[d', d'']$. The rate tenor is $\tau = d'' - d'$.

Then NPV of the fixed leg is calculated as follows:

$$
\NPV_\fixed(d;K) = N \cdot K \cdot \sum_{i=1}^p P\left(d, d_i^\fixed \right)
$$

and the NPV of the floating leg is calculated as follows:

$$
\NPV_\float(d) = N \cdot \sum_{j=1}^q F\left(d, d_{j-1}^\float, d_j^\float \right) \cdot \frac{d_j^\float -  d_{j-1}^\float}{360} \cdot P\left(d, d_j^\float \right)
$$

Therefore, the NPV of the swap (seen from the point of view of the counterparty which recieves the floating leg) is

$$
\NPV(d;K) = \NPV_\float(d) - \NPV_\fixed(d; K)
$$

For reasons which will become apparent later, it's actually more convenient to express the NPV of an IRS as a function of the fair value fixed rate $S$ of that IRS, also known as the swap rate. $S$ is the value of $K$ which makes $NPV(d) = 0$.

On the basis of the previous expressions, we can easily calculate $S$ as:

$$
S = \frac{\sum_{j=1}^q F\left(d, d_{j-1}^\float, d_j^\float \right) \cdot \frac{d_j^\float -  d_{j-1}^\float}{360} \cdot P\left(d, d_j^\float \right)}{\sum_{i=1}^p P\left(d, d_i^\fixed \right)}
$$

Once we've calculated $S$, we can express the NPV of an IRS as follows:

$$
\begin{align}
\NPV(d;K)
&= \NPV_\float(d) - \NPV_\fixed(d; K) \\
&= \underbrace{\NPV_\float(d) - \NPV_\fixed(d; S)}_{=0} + \NPV_\fixed(d; S) - \NPV_\fixed(d; K) \\
&= N \cdot (S-K) \cdot \underbrace{\sum_{i=1}^p P\left(d, d_i^\fixed \right)}_{\text{'annuity'}}
\end{align}
$$

In [2]:
from curve_data import pricing_date, discount_curve, libor_curve

In [3]:
discount_curve.df(date(2017, 1, 1))

1.0003778376026289

In [4]:
libor_curve.forward_rate(date(2017, 1, 1))

0.010002671232876712

In [5]:
class InterestRateSwap(object):
    
    def __init__(self, start_date, notional, fixed_rate,
                 tenor_months, maturity_years):
        
        self.notional = notional
        self.fixed_rate = fixed_rate
        
        self.fixed_leg_dates = generate_swap_dates(
            start_date,
            12 * maturity_years,
            12
        )
        
        self.floating_leg_dates = generate_swap_dates(
            start_date,
            12 * maturity_years,
            tenor_months
        )
        
    def annuity(self, discount_curve):
        
        a = 0
        
        for i in range(1, len(self.fixed_leg_dates)):
            a += discount_curve.df(self.fixed_leg_dates[i])
            
        return a
    
    def swap_rate(self, discount_curve, libor_curve):
        
        s = 0
        
        for j in range(1, len(self.floating_leg_dates)):
            F = libor_curve.forward_rate(
                self.floating_leg_dates[j-1]
            )
            
            tau = (
                self.floating_leg_dates[j] -
                self.floating_leg_dates[j-1]
            ).days / 360
            
            P = discount_curve.df(
                self.floating_leg_dates[j]
            )
            
            s += F * tau * P
            
        return s / self.annuity(discount_curve)
    
    def npv(self, discount_curve, libor_curve):
        S = self.swap_rate(discount_curve, libor_curve)
        A = self.annuity(discount_curve)
        return self.notional * (S - self.fixed_rate) * A

In [6]:
irs = InterestRateSwap(pricing_date, 1e6, 0.05, 6, 4)
irs.npv(discount_curve, libor_curve)

-160129.63585748823

In [7]:
irs.swap_rate(discount_curve, libor_curve)

0.010254490656171655

## Interest rate swaptions

Swaptions are the equivalent of European options for the interest rate markets. They give the option holder the right but not the obligation, at the exercise date $\def\ex{\mathrm{ex}} d_\ex$, to enter into an IRS at a pre-determined fixed rate.

Clearly the option holder will only choose to do this if the NPV of the underlying swap at $d_\ex$ is positive - looking at the expression for the NPV of the IRS in terms of the swap rate $S$ therefore, we can see that the payoff of the swaption is

$$
N \cdot \max(0, S(d_\ex) - K) \cdot \sum P\left(d_\ex, d_i^\fixed \right)
$$

To evaluate the NPV of this payoff, we'll use the following formula, which is a generalization of the Black-Scholes-Merton formula applied to swaptions:

$$
\def\N{\mathcal{N}}
\NPV = \phi A \left[ S \N(\phi d_+) - K \N(\phi d_-) \right]
$$
where
$$
\begin{align}
d_{\pm} &= \frac{\log(\tfrac S K) \pm \tfrac 1 2 \sigma^2 T}{\sigma \sqrt T} \\
A &= \sum_{i=1}^p P\left(d, d_i^\fixed \right)
\end{align}
$$

In [8]:
from scipy.stats import norm  # norm.cdf is the Gaussian cumulative distribution function

sigma = 0.07

irs = InterestRateSwap(date(2018, 1, 1), 1e6, 0.01, 6, 4)

exercise_date = date(2017, 6, 1)

In [9]:
import math

A = irs.annuity(discount_curve)
S = irs.swap_rate(discount_curve, libor_curve)

T = (exercise_date - pricing_date).days / 365

d1 = (math.log(S/irs.fixed_rate) + 0.5 * sigma**2 * T) / (sigma * T**0.5)
d2 = (math.log(S/irs.fixed_rate) - 0.5 * sigma**2 * T) / (sigma * T**0.5)

npv = irs.notional * A * (S * norm.cdf(d1) - irs.fixed_rate * norm.cdf(d2))

# print a phrase presenting the results
print("Swaption NPV: %.2f EUR" % npv)

Swaption NPV: 1839.21 EUR


An alternative way of performing this pricing is via a Monte-Carlo simulation. We start from the current swap rate $S(d)$ evaluated at the pricing date $d$, and assume that it follows a log-normal stochastic process, so its distribution at $d_\ex$ is  $S(d_\ex) = S(d) \exp\left\{ -\tfrac 1 2 \sigma^2 T + \sigma \sqrt T \epsilon \right\}$ where $\epsilon \sim N(0, 1)$. To perform the simulation, we sample the distribution to calculate a large number of scenarios for $S(d_\ex)$, we evaluate the underlying swap's NPV at the expiry date, and consequently the swaption's payoff, and take the average of these values.

In [10]:
# we'll need numpy.mean and numpy.std to calculate the average and standard deviation of a list of values
import numpy as np

# the 'numpy.random.normal' function returns a random sample from the standard Gaussian distribution
from numpy.random import normal

# define the number of Monte Carlo scenarios
n_scenarios = 100000

# initialize a variable to store the discounted payoff of the swaption in each Monte Carlo scenario
discounted_payoffs = []

# perform the Monte Carlo simulation - loop over each scenario
for i_scenario in range(n_scenarios):
    
    # simulate the swap rate in this scenario
    S_simulated = S * math.exp(-0.5 * sigma * sigma * T + sigma * math.sqrt(T) * normal())
    
    # calculate the swap NPV in this scenario
    swap_npv = irs.notional * (S_simulated - irs.fixed_rate) * A
    
    # add the discounted payoff of the swaption, in this scenario, to the list
    discounted_payoffs.append(max(0, swap_npv))
    
    # Note that this is not *strictly speaking* the correct way of calculating the
    # value, the reason being that one should calculate the swap NPV at the expiry date
    # of the swaption, apply the payoff function max(0, ...) and *then* discount from the
    # expiry date to today.
    #
    # However, it's simpler to calculate it as above and it doesn't make any difference to
    # the result, since DiscountFactor * max(0, SwapNPVAtExpiry) == max(0, DiscountFactor * SwapNPVAtExpiry).
    
# calculate the NPV of the swaption by taking the average of the discounted payoffs across all the scenarios
npv_mc = np.mean(discounted_payoffs)

# calculate the Monte Carlo error estimate for 'npv_mc'
# this will give us a 99% confidence interval for the calculated value
npv_error = 3 * np.std(discounted_payoffs) / math.sqrt(n_scenarios)

# print a phrase presenting the results
print("Swaption NPV: %.2f EUR (+/- %.2f EUR with 99%% confidence)" % (npv_mc, npv_error))

Swaption NPV: 1839.59 EUR (+/- 16.61 EUR with 99% confidence)


The NPV calculated via the Black-Scholes-Merton formula falls within the confidence interval produced by the Monte Carlo simulation, so we can assert that the two methods are in agreement.